# <center>Preprocessing Code for the </center>
# <center>Department of Homeland Security </center>
# <center>Passenger Screening Algorithm Challenge.</center>

# General imports and initializations

In [ ]:
import math
import os
import pdb
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import datetime
import csv

from scipy import ndimage
from scipy.signal import medfilt
from scipy.signal import savgol_filter
from scipy.signal import resample
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.filters import median_filter
from scipy.ndimage.morphology import binary_fill_holes
from scipy.ndimage.morphology import binary_dilation
from scipy.misc import imsave
from scipy.ndimage import imread
from copy import deepcopy
from scipy import linalg
from scipy.interpolate import interp1d
from skimage.transform import resize
from sklearn.preprocessing import binarize

In [ ]:
!pip3 install Cython
%load_ext cython

In [ ]:
%matplotlib inline

In [ ]:
%pdb off

In [ ]:
plt.style.use('classic')

# Project-specific imports and initializations

In [ ]:
# Get project functions
from CompetitionFileIOFunctions import initRootFolders, initLog, log, filenames, loadFile

In [ ]:
# Init root folders
inCloud = True
if inCloud:
    # Working in the cloud.
    # Read scans from the bucket and save results locally:
    initRootFolders(bucketName='kaggle_passenger_screening123407', localIOPath='')
else:
    # Working on a desktop
    # Don't bother with the bucket, but set a constant local IO path to sidestep versioning
    initRootFolders(
        bucketName='', 
        localIOPath='/media/qwerty/science/science data/2017-10-18 Kaggle passenger screening/'
    )

# Name the input/output folders
# cloud/ and local/ refer to locations defined in bucketName and localIOPath (above)
scanDir1 = "cloud/stage1_a3d/"
scanDir2 = "cloud/stage2_a3d/"
embeddedDir1 = "local/embedded2D/stage1/"
embeddedDir2 = "local/embedded2D/stage2/"
logDir = "local/log/"

In [ ]:
# Initialize log file
initLog(logDir, 'embed')

In [ ]:
# Threshold for finding body region in 3D
threshold3D = .0002

# Read file data

In [ ]:
inputFiles = filenames(scanDir1)

In [ ]:
fileNum = 1
inputFile = inputFiles[fileNum]

In [ ]:
####################################################
log('+++++ Read file ' + str(fileNum) + ' +++++')
####################################################
# Note: title comments like the one above indicate 
# notebook cells that are copy/pasted to the main 
# processing loop at the bottom of the notebook.
data = loadFile(inputFile)

# Threshold

In [ ]:
####################################################
log('Threshold')
####################################################
thresholded = np.clip(data, threshold3D, 100000) - threshold3D

In [ ]:
print('Total reflectivity: ', np.sum(thresholded))

# Average along axii to get 2D mean images

In [ ]:
####################################################
log('Average along axii to get 2D mean images')
####################################################
# Summed images will have yz, xz, xy axes, respectively
means2D = list([thresholded.mean(axis=i) for i in range(3)])
mean2DAxii = [('y', 'z'), ('x','z'), ('x','y')]

In [ ]:
fig, ax = plt.subplots(3, figsize=(30,30))
for i in range(3):
    ax[i].imshow(means2D[i], cmap = 'viridis', interpolation = 'nearest')

# Get center of mass

In [ ]:
####################################################
log('Get center of mass')
####################################################
# Could do this on the 3D image, but it's much faster on 2D images 
xCenter, zCenter = np.round(ndimage.measurements.center_of_mass(means2D[1])).astype(int)
yCenter, dmy = np.round(ndimage.measurements.center_of_mass(means2D[0])).astype(int)

In [ ]:
print('xCenter:', xCenter, '    yCenter:', yCenter, '    zCenter:', zCenter)

# Get end of trunk z-range (top of shoulders)

In [ ]:
def firstOn(dat, threshold=0.33):
    filt = medfilt(dat, 11)
    thresh = threshold * max(filt)
    binarized = filt>thresh
    first = np.argmax(binarized)
    return first

In [ ]:
def firstOff(dat, threshold=0.33):
    filt = medfilt(dat, 11)
    thresh = threshold * max(filt)
    binarized = filt<=thresh
    first = np.argmax(binarized)
    return first

In [ ]:
####################################################
log('Get end of trunk z-range (top of shoulders)')
####################################################
zStart, zEnd = (int(zCenter * 1.2), int(zCenter * 1.7))
if zEnd > data.shape[2]: zEnd = data.shape[2]

xStart = int(xCenter - .25 * zCenter)
xEnd = int(xCenter + .25 * zCenter)

midStrip = means2D[1][xStart:xEnd, zStart:zEnd]

off = midStrip < 0.00000001
numOff = np.sum(off, axis=0)

trunkEnd = firstOn(numOff, .15)
trunkEnd += zStart

In [ ]:
fig, ax = plt.subplots(1, figsize=(3,3),facecolor='white')
ax.imshow(off, cmap = 'viridis', interpolation = 'nearest')
ax.set_xlabel('z')
ax.set_ylabel('x')

fig, ax = plt.subplots(1,facecolor='white')
ax.plot(numOff)
ax.set_xlabel('z')
ax.set_ylabel('Number of empty pixels')

print(trunkEnd)

# Get start of trunk z-range (bottom of groin)

In [ ]:
####################################################
log('Get start of trunk z-range (bottom of groin)')
####################################################
zStart = int(0.5 * zCenter)
zStrip = np.mean(thresholded[:, yCenter:, zStart:zCenter], axis=0)
zStrip = gaussian_filter(zStrip, (5,5))
buttProfile = np.array([firstOff(x, .05) for x in np.transpose(zStrip)])
buttProfile = median_filter(buttProfile, 5)
buttProfileDer = savgol_filter(buttProfile, 11, 2, deriv=1, mode='nearest')
trunkStart = zStart + np.argmax(buttProfileDer)

In [ ]:
fig, ax = plt.subplots(1, figsize=(5,5))
ax.imshow(zStrip, cmap = 'viridis', interpolation = 'nearest')

fig, ax = plt.subplots(1, figsize=(12,3.5), facecolor='white')
ax.plot(buttProfile)
ax.set_xlabel('z')
ax.set_ylabel('Buttock profile')

fig, ax = plt.subplots(1, figsize=(12,3.5), facecolor='white')
ax.plot(buttProfileDer)
ax.set_xlabel('z')
ax.set_ylabel('Derivative of buttock profile w.r.t. z')

print(trunkStart)

# Erase head

In [ ]:
%%cython 
#--annotate
cimport cython
import math
import numpy as np
from cython.view cimport array as cvarray

cdef extern from "math.h":
    int floor(float x)
    
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def eraseHead(float [:,:,:] arr, float centerX, float centerZ, float radiusX, float radiusZ):
    """Replace head region with zeros"""
    
    cdef int nx = arr.shape[0]
    cdef int ny = arr.shape[1]
    cdef int nz = arr.shape[2]
    cdef int x, y, z

    cdef float [:,:,:] out = np.array(arr, dtype=np.float32)

    cdef float radiusX2 = radiusX**2
    cdef float radiusZ2 = radiusZ**2
    cdef float r

    cdef int xStart = floor(centerX - radiusX)
    cdef int xEnd = floor(centerX + radiusX)
    cdef int zStart = floor(centerZ - radiusZ)
    cdef int zEnd = floor(centerZ + radiusZ)

    xStart = 0 if xStart<0 else xStart
    xEnd = nx if xEnd>nx else xEnd

    zStart = 0 if zStart<0 else zStart
    zEnd = nz if zEnd>nz else zEnd

    for x in range(xStart, xEnd):
        
        for z in range(zStart, zEnd):
            
            r = (x - centerX)**2 / radiusX2 + (z - centerZ)**2 / radiusZ2
            
            if r < 1:

                out[x,:,z] = 0.

    return np.array(out, dtype=np.float32)

In [ ]:
####################################################
log('Erase head')
####################################################
# Head position, size
headCenterZ = trunkEnd * 1.12
headHeight = trunkEnd * .15
headWidth = trunkEnd * .13

# Get center X for head, which may be shifted relative to body
headStripStartZ = math.floor(trunkEnd * 1.05)
headStripEndZ = math.floor(trunkEnd * 1.1)
headStripStartX = math.floor(xCenter - trunkEnd*.18)
headStripEndX = math.floor(xCenter + trunkEnd*.18)
headStripMean = np.mean(thresholded[headStripStartX:headStripEndX, :, headStripStartZ:headStripEndZ], axis=(1,2))
middle = math.floor(len(headStripMean)/2)
rightMost = firstOff(headStripMean[middle:0:-1], .05)
leftMost = firstOff(headStripMean[middle:-1], .05)
headOffsetX = math.floor((leftMost-rightMost)/2)

# Erase head by deleting cylindrical region
thresholded = eraseHead(thresholded, xCenter + headOffsetX, headCenterZ, headWidth, headHeight)
data = eraseHead(data, xCenter + headOffsetX, headCenterZ, headWidth, headHeight)

# Recalculate averages along axii
means2D = list([thresholded.mean(axis=i) for i in range(3)])
mean2DAxii = [('y', 'z'), ('x','z'), ('x','y')]

In [ ]:
print(headStripStartZ, headStripEndZ, headStripStartX, headStripEndX)

plt.plot(headStripMean)

fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(np.mean(thresholded, axis=1), cmap = 'viridis', interpolation = 'nearest')

# Get waffle stacks

In [ ]:
%%cython 
#--annotate
cimport cython
import math
import numpy as np
from cython.view cimport array as cvarray

cdef extern from "math.h":
    int floor(float x)
    
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)

def rotateArray(
    float [:,:,:] arrayXYZ, 
    int [:] xRange, 
    int [:] yRange, 
    int [:] zRange,
    float [:] origin, 
    float [:,:] basis, 
    int radius,
    int length,
    float threshold):
  
    """
    Given a volumetric scan, rotates it so that the given basis becomes the standard basis.
    
    Input:
      arrayXYZ:  The volumetric scan to be rotated
      xRange, yRange, zRange: The ranges of the input array to be copied
      origin: The origin for the rotation
      basis: An orthogonal basis for the rotation.  Row vectors become standard basis axii after the rotation.
      radius: The output array's height, width and depth will be 2*radius+1
      length: Voxels will be skipped if their projection on the last basis vector is greater then this
      threshold: Voxels will be skipped if they are less than this value.
      
    Output:
      Rotated array
    """
    
    cdef float [:,:,:,:] out = np.zeros(shape=(radius*2, radius*2, length, 2), dtype=np.float32)

    cdef int xStart = min(xRange)
    cdef int xEnd = max(xRange)
    cdef int yStart = min(yRange)
    cdef int yEnd = max(yRange)
    cdef int zStart = min(zRange)
    cdef int zEnd = max(zRange)

    if zEnd > arrayXYZ.shape[2]: zEnd = arrayXYZ.shape[2]
      
    # Basis matrix:  rows (u,v,w), columns (x,y,z)
    # It's stored as a bunch of floats to maximize the chance they'll end up in CPU registers
    # I'm avoiding Numpy since vectorization won't help much with tiny matrices and vectors
    cdef float bux = basis[0,0]
    cdef float buy = basis[0,1]
    cdef float buz = basis[0,2]
    cdef float bvx = basis[1,0]
    cdef float bvy = basis[1,1]
    cdef float bvz = basis[1,2]
    cdef float bwx = basis[2,0]
    cdef float bwy = basis[2,1]
    cdef float bwz = basis[2,2]

    # xyMask is used to skip vertical slices that are empty (I.e. below threshold)
    cdef float [:,:] xyMask = np.mean(arrayXYZ, axis=2)
    cdef float xyThreshold = threshold / 100
    
    cdef int x,y,z,ou,ov,ow
    cdef float val, ox, oy, oz, norm
    
    # Loop over voxels, accumulate 2D embedding
    for x in range(xStart, xEnd):
        for y in range(yStart, yEnd):
            
            # Skip empty z-slices
            if xyMask[x,y] < xyThreshold: continue
                
            for z in range(zStart, zEnd):
                
                # Skip empty voxels
                val = arrayXYZ[x,y,z]
                if val < threshold: continue
                    
                ox, oy, oz = (x-origin[0], y-origin[1], z-origin[2])
                 
                # Transform (ox,oy,oz) to uvw basis
                ou, ov, ow = (
                    floor(bux*ox + buy*oy + buz*oz) + radius,
                    floor(bvx*ox + bvy*oy + bvz*oz) + radius,
                    floor(bwx*ox + bwy*oy + bwz*oz))

                if ou>=0 and ou<radius*2 and ov>=0 and ov<radius*2 and ow>=0 and ow<length:
                    out[ou,ov,ow,0] += 1
                    out[ou,ov,ow,1] += val

    # Normalize by number of voxels
    for ou in range(0, 2* radius):
        for ov in range(0, 2* radius):
            for ow in range(0, length):
                norm = out[ou, ov, ow, 0]
                if norm > .00000001:
                    out[ou, ov, ow, 1] /= norm

    return np.array(out[:,:,:,1], dtype=np.float32)

In [ ]:
def getCenterAndRadius(xIn, thresholdIn):
    """
    Finds the left and right edge of an object in xIn.
    Returns the center and radius.
    """
    
    x = medfilt(xIn, 11)
    threshold = np.max(x)*thresholdIn
    binarized = x>threshold
    first = np.argmax(binarized)
    last = len(x) - np.argmax(np.flipud(binarized))
    
    return (math.floor((first+last)/2), (last-first)/2)

In [ ]:
def makeWaffles(arrayXYZ, xRange, yRange, zRange, startIn, endIn, basis, radius, threshold, filterWidth):
    """
    Given a volumetric scan arrayXYZ that contains a tube-like surface (a leg, for instance),
    this function finds a stack of ellipses that approximates the surface.
    
    Input:
      xRange, yRange, zRange: Voxels outside these ranges are ignored
      startIn: A pre-estimate of the center of the first ellipse
      endIn: A pre-estimate of the center of the last ellipse
      basis: An orthogonal basis for the cylindrical coordinate transformation.  
          The last row vector corresponds to the axial coordinate.
          The first two row vectors define the plane for the ellipses.
          The ellipse axii will be aligned to the first two row vectors (not rotated in the plane).
      radius: radius for the array rotation, if required
      threshold: threshold for the array rotation, if required
      filterWidth: Used when filtering the lists of ellipse centers and axii lengths
      
    Output:
      A "waffle stack":  a dictionary containing a starting point, basis, ellipse parameters, etc.
    """
  
    if type(startIn)==type(0):
        # The start and end positions are z positions.
        # Get x and y positions
        # This should only be done when the basis is the standard basis 
        # and waffles are stacked along z.
        zSlice = arrayXYZ[xRange[0]:xRange[1], yRange[0]:yRange[1], startIn]
        startX, dmy = getCenterAndRadius(np.mean(zSlice, axis=1), 0.1)
        startY, dmy = getCenterAndRadius(np.mean(zSlice, axis=0), 0.1)
        start = np.array([startX+xRange[0], startY+yRange[0], startIn], dtype=np.int32)

        zSlice = arrayXYZ[xRange[0]:xRange[1], yRange[0]:yRange[1], endIn]
        endX, dmy = getCenterAndRadius(np.mean(zSlice, axis=1), 0.1)
        endY, dmy = getCenterAndRadius(np.mean(zSlice, axis=0), 0.1)
        end = np.array([endX+xRange[0], endY+yRange[0], endIn], dtype=np.int32)
        
        length = endIn - startIn
        
    else:
        # We already have x,y,z coordinates for start and end positions. 
        start, end = (startIn, endIn)
        length = int(math.floor(np.linalg.norm(end-start)))
        
    # Rotate scane into specified UVW basis
    arrayUVW = rotateArray(
        arrayXYZ, 
        np.array(xRange, dtype=np.int32), 
        np.array(yRange, dtype=np.int32), 
        np.array(zRange, dtype=np.int32), 
        np.array(start, dtype=np.float32), 
        np.array(basis, dtype=np.float32), 
        radius, length, threshold
    )
    #return arrayUVW

    # Initialize waffle offsets and widths
    uOffsets = np.empty(length, np.float64)
    uWidths = uOffsets.copy()
    vOffsets = uOffsets.copy()
    vWidths = uOffsets.copy()

    # Get waffle widths and offsets from center
    for i in range(0, length):
        zSlice = arrayUVW[:,:,i]
        uOffsets[i], uWidths[i] = getCenterAndRadius(zSlice.mean(axis=1), 0.05)
        vOffsets[i], vWidths[i] = getCenterAndRadius(zSlice.mean(axis=0), 0.05)
    uOffsets -= radius
    vOffsets -= radius
    #plt.plot(uOffsets)
    
    # Reduce impulse noise
    uOffsets = medfilt(uOffsets, 5)
    uWidths = medfilt(uWidths, 5)
    vOffsets = medfilt(vOffsets, 5)
    vWidths = medfilt(vWidths, 5)

    # Smooth
    filtLength = math.floor(length * filterWidth/2)*2+1
    filtLength = filtLength if filtLength>=5 else 5
    uOffsets = savgol_filter(uOffsets, filtLength, 1)
    uWidths = savgol_filter(uWidths, filtLength, 1)
    vOffsets = savgol_filter(vOffsets, filtLength, 1)
    vWidths = savgol_filter(vWidths, filtLength, 1)

    # Return waffle stack
    return {
        "rangeXYZ": np.array([xRange, yRange, zRange], dtype=np.int32),
        "basis": np.array(basis, dtype=np.float32),
        "start": np.array(start, dtype=np.float32),
        "uOffsets": np.array(uOffsets, dtype=np.float32),
        "uWidths": np.array(uWidths, dtype=np.float32),
        "vOffsets": np.array(vOffsets, dtype=np.float32),
        "vWidths": np.array(vWidths, dtype=np.float32),
        "butter": 'LandOfLakes',
        "syrup": 'Maple'
    }

# Get leg and trunk waffle stacks

In [ ]:
####################################################
log('Get leg and trunk waffle stacks')
####################################################

# Right leg
end = int(1.27 * trunkStart)
rightLeg = makeWaffles(
    thresholded, 
    (0, xCenter), 
    (0, thresholded.shape[1]), 
    (0, end), 
    0, end, np.eye(3), 200, 0.00001, .16
)

# Left leg
end = int(1.27 * trunkStart)
leftLeg = makeWaffles(
    thresholded, 
    (xCenter, thresholded.shape[0]), 
    (0, thresholded.shape[1]), 
    (0, end), 
    0, end, np.eye(3), 200, 0.00001, .16
)

# Trunk
start = int(.95 * trunkStart)
end = int(trunkEnd)
trunk = makeWaffles(
    thresholded, 
    (0, thresholded.shape[0]), 
    (0, thresholded.shape[1]), 
    (start, end), 
    start, end, np.eye(3), 250, 0.00001, .16
)

# Get arm waffles

The following function is a dirty kitchen sink.  There's also a problem with the elbows:  the way they're done now, waffles near an elbow are 90 degrees from the direction they should be.  The ugly code and bad elbow embedding could be fixed by implementing curvy waffle stacks (waffles not parallel).  This would require changing all the 2D embedding code - about 1 week of work.  Elbow embedding would improve and the following function could be much cleaner.

In [ ]:
def getArmJointCoordinates(thresholdedIn, means2DIn, xCenter, armpitZ, leftSide):

    ####################################    
    # Get data for left or right side 
    ####################################
    if leftSide:
        thresholded = np.flipud(thresholdedIn[xCenter:-1]) 
        xzSlice = np.flipud(means2DIn[1][xCenter:-1,armpitZ:thresholded.shape[2]])
    else:
        thresholded = thresholdedIn[0:xCenter]
        xzSlice = means2D[1][0:xCenter,armpitZ:thresholded.shape[2]]

    ####################################
    # Get fingertip height
    ####################################
    zStrip = np.mean(xzSlice, axis=0)
    fingertipZ = int(firstOn(np.flipud(zStrip), .05))
    fingertipZ = thresholded.shape[2] - fingertipZ - 1
    
    ####################################
    # Get arm waffles
    ####################################
    arm = makeWaffles(
        thresholded, 
        (0, thresholded.shape[0]), (0, thresholded.shape[1]), (armpitZ, fingertipZ),
        armpitZ, fingertipZ, np.eye(3), 200, .00001, .1
    )
    
    ####################################
    # Get shoulder position
    ####################################
    shoulderPos = arm["start"]
    shoulderPos[0] -= armpitZ * .05
    
    ####################################
    # Estimate elbow position
    ####################################
    armLength = len(arm["uOffsets"])
    elbowZ = math.floor(0.4 * armLength)

    # Get second derivative
    filterWidth = math.floor(0.15 * armLength)
    d2x = gaussian_filter(arm["uOffsets"], (filterWidth), order=2, mode="nearest")

    # Improve estimate of elbow height if arm is bent
    if d2x[elbowZ] > .01:
        elbowIndicator = -arm["uOffsets"] + 25*70*d2x
        middleThird = [math.floor(armLength/3), math.floor(armLength*2/3)]
        elbowZ = np.argmax(elbowIndicator[middleThird[0]:middleThird[1]])
        elbowZ = elbowZ + middleThird[0]

    # Get full-scan coordinates
    elbowX = math.floor(arm['start'][0] + arm['uOffsets'][elbowZ])
    elbowY = math.floor(arm['start'][1] + arm['vOffsets'][elbowZ])
    elbowZ += armpitZ
    elbowPos = np.array([elbowX, elbowY, elbowZ])
    
    ####################################
    # Estimate wrist position
    ####################################    
    numBins = 50
    wristAt = 0.55
    wristDel = 0.1

    # Get full-scan XYZ coordinates of voxels in the forearm region
    forearmImg = thresholded[:, :,elbowZ:fingertipZ]
    forearmCoords = np.argwhere(forearmImg>0.00000001).astype(np.float32)
    forearmCoords[:,2] += elbowZ

    # Get vector from elbow to forearm center of mass
    forearmCoM = np.mean(forearmCoords, axis=0)
    forearmDirection = forearmCoM - elbowPos
    forearmLength = np.linalg.norm(forearmDirection)
    forearmDirection /= forearmLength
    
    # Project forearm coordinates on forearm vector
    forearmCoords -= elbowPos
    forearmW = np.dot(forearmCoords, forearmDirection)
    forearmR = np.linalg.norm(forearmCoords - np.outer(forearmW, forearmDirection), axis=1)

    # Build trajectory in wr space
    orderW = np.argsort(forearmW)
    trajectory = np.zeros(shape=(numBins,2), dtype=np.float32)
    mass = np.zeros(shape=numBins, dtype=np.float32)
    binNumberFromW = numBins / (3 * forearmLength)
    
    for i in range(len(orderW)):
        w = forearmW[orderW[i]]
        r = forearmR[orderW[i]]
        binIdx = math.floor(w * binNumberFromW)
        if binIdx > numBins-1 or binIdx < 0:  continue
        mass[binIdx] += 1
        trajectory[binIdx] += [w,r]
    
    # Normalize by mass and delete bins without mass
    endBin = -1
    gotOne = False
    for i in range(numBins):
        if mass[i] > 0.00000001:
            trajectory[i] /= mass[i]
            gotOne = True
        elif gotOne: 
            endBin = i
            break
    trajectory = trajectory[0:endBin]
    mass = mass[0:endBin]

    # Get length of trajectory 
    trajectoryDels = np.linalg.norm(trajectory[1:-1] - trajectory[0:-2], axis=1)
    trajectoryLength = sum(trajectoryDels)

    # Get middle of trajectory
    wristPos = trajectoryLength * wristAt
    trajectoryAccum = 0
    for i in range(len(trajectory)):
        trajectoryAccum += trajectoryDels[i]
        if trajectoryAccum > wristPos:
            wristPos = i
            break
    wristW = trajectory[wristPos, 0]

    # Get mean XYZ position of voxels near wrist
    startW = wristW - wristDel * forearmLength
    endW = wristW + wristDel * forearmLength
    wristPos = np.array([0,0,0], dtype=np.float32)
    wristMass = 0
    for i in range(len(forearmCoords)):
        if forearmW[i] > startW and forearmW[i] < endW:
            wristPos += forearmCoords[i]
            wristMass += 1
    wristPos /= wristMass
    wristPos += elbowPos

    ####################################
    # Return results
    ####################################

    # Flip x-coordinate
    if leftSide:
        shoulderPos[0] = thresholdedIn.shape[0] - shoulderPos[0]
        elbowPos[0] = thresholdedIn.shape[0] - elbowPos[0]
        wristPos[0] = thresholdedIn.shape[0] - wristPos[0]

    return (shoulderPos, elbowPos, wristPos, fingertipZ)

In [ ]:
def getArmSegmentBasis(start, end):
    w = end - start
    length = np.linalg.norm(w)
    w /= length

    u = np.cross(w, [0,0,1])
    u /= np.linalg.norm(u)

    v = np.cross(u, w)
    v /= np.linalg.norm(v)
    
    return np.array([u,v,w], dtype=np.float32)

In [ ]:
####################################################
log('Get arm waffle stacks')
####################################################

# Guess armpit height
armpitZ = int(trunkStart + 0.9 * (trunkEnd - trunkStart))

# Right arm
shoulderPos, elbowPos, wristPos, fingertipZ = getArmJointCoordinates(thresholded, means2D, xCenter, armpitZ, False)

# Right bicep
rightBicep = makeWaffles(
    thresholded,
    (0, xCenter), (0, thresholded.shape[1]), (math.floor(shoulderPos[2]*.9), elbowPos[2]+20),
    shoulderPos, elbowPos, getArmSegmentBasis(shoulderPos, elbowPos), 80, 0.00001, .7
)

# Right forearm
rightForearm = makeWaffles(
    thresholded,
    (0, xCenter), (0, thresholded.shape[1]), (elbowPos[2]-20, wristPos[2]+20),
    elbowPos, wristPos, getArmSegmentBasis(elbowPos, wristPos), 60, 0.00001, .9
)

# Left arm
shoulderPos, elbowPos, wristPos, fingertipZ = getArmJointCoordinates(thresholded, means2D, xCenter, armpitZ, True)

# Left bicep
leftBicep = makeWaffles(
    thresholded,
    (xCenter, thresholded.shape[0]), (0, thresholded.shape[1]), (math.floor(shoulderPos[2]*.9), elbowPos[2]+20),
    shoulderPos, elbowPos, getArmSegmentBasis(shoulderPos, elbowPos), 80, 0.00001, .7
)

# Left forearm
leftForearm = makeWaffles(
    thresholded,
    (xCenter, thresholded.shape[0]), (0, thresholded.shape[1]), (elbowPos[2]-20, wristPos[2]+20),
    elbowPos, wristPos, getArmSegmentBasis(elbowPos, wristPos), 60, 0.00001, .9
)

# Plot segment waffle stacks

This does not go in the main processing loop.  It's only used here, in the notebook, for debugging and development.

In [ ]:
def drawSegment(array, segment):
    length = len(segment["uOffsets"])
    start = segment["start"]
    basis = segment["basis"]
    out = np.array(array)
    pos = []
    wNorm = np.linalg.norm(basis[2])
    
    for w in range(length):

        uo = segment["uOffsets"][w]
        uw = segment["uWidths"][w]
        vo = segment["vOffsets"][w]
        vw = segment["vWidths"][w]

        wPos = (w / wNorm**2)
        pos.append(start + np.dot([uo, vo, wPos], basis))
        pos.append(start + np.dot([uo+uw, vo, wPos], basis))
        pos.append(start + np.dot([uo-uw, vo, wPos], basis))
        pos.append(start + np.dot([uo, vo+vw, wPos], basis))
        pos.append(start + np.dot([uo, vo-vw, wPos], basis))

    pos = np.array(np.floor(pos), dtype=np.int32)
    
    out[pos[:,0], pos[:,1], pos[:,2]] = .005
    
    return out

In [ ]:
s = thresholded
s = drawSegment(s, trunk)
s = drawSegment(s, rightLeg)
s = drawSegment(s, leftLeg)
s = drawSegment(s, rightBicep)
s = drawSegment(s, leftBicep)
s = drawSegment(s, rightForearm)
s = drawSegment(s, leftForearm)

# Front
xz = np.mean(s, axis=1)
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(xz, cmap = 'viridis', interpolation = 'nearest')

# Right
yz = np.mean(s[0:xCenter+20,:,:], axis=0)
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(yz, cmap = 'viridis', interpolation = 'nearest')

# Left
yz = np.mean(s[xCenter-20:-1,:,:], axis=0)
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(yz, cmap = 'viridis', interpolation = 'nearest')

# Forearm may be bad due to movement during scan

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(thresholded[:,:,560], cmap = 'viridis', interpolation = 'nearest')

# Embed in cylindrical coordinates

This section and the next few sections contain code that will be placed in the function getSegmentImage(), instead of going directly in the main processing loop.

In [ ]:
%%cython 
#--annotate
cimport cython
import math
import numpy as np
from cython.view cimport array as cvarray

cdef extern from "math.h":
    float atan2(float x, float y)

cdef extern from "math.h":
    int floor(float x)

cdef extern from "math.h":
    float sqrt(float x)

cdef extern from "math.h":
    float fabs(float x)
    
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def toCylindricalCoordinates( 
    float threshold,
    float [:,:,:] arrayXYZ,
    int [:,:] rangeXYZ,
    float [:,:] basis,
    float [:] startXYZ,
    float [:] uOffsets,
    float [:] uWidths,
    float [:] vOffsets,
    float [:] vWidths,
    int nTheta,
    int nRadius,
    float [:] radiusRange
    ):

  
    """
    Get a cylindrical coordinate transformation of a portion of the input arrayXYZ.
    The input array should have a tube-like object (a leg for instance).
    The output array will contain the tube-like object, "unrawapped" by a cylindrical coordinate transformation.
    
    Notes on geometry:
    Argument 'startXYZ' is the origin of the ray along which waffles are stacked.  It's in the standard basis.
    Argument 'basis' contains basis vectors u^hat, v^hat, w^hat.  
    Arguments 'uOffsets' and 'vOffsets' contain offsets for the waffles, in the u+v subspace
    Algorithm steps:
        1: Subtract startXYZ from a voxel's xyz coordinates
        2: Transform xyz coordinates to uvw coordinates
        3: Subtract the offset in u+v space. This yields cylU and cylV.
        4: Get cylindrical coordinates:
            cylR = sqrt(cylU**2 + cylV**2)
            cylTheta = atan2(cylV/vWidth, cylU/uWidth)
            cylW = w coordinate from step 2
        5: Add voxel's value to output array
    The output output array is indexed by spatial coordinate (cylTheta, cylR, cylW)
    """
    
    # wLen is the number of waffles used in the cylindrical coordinate embedding
    cdef int wLen = len(uOffsets)

    # Variable out is a 2D image with 3 planes. 
    # Plane 0: total intensity (0th moment)
    # Plane 1: intensity-weighted distance (1st moment)
    # Plane 2: 2nd moment
    # It is indexed by spatial coordinate (cylW, cylTheta)
    cdef float [:,:,:,:] out = cvarray(shape=(nRadius, nTheta, wLen, 2), itemsize=sizeof(float), format="f")
    out[:,:,:] = 0.

    # xyMask is used to skip vertical slices that are empty (I.e. below threshold)
    cdef float [:,:] xyMask = np.mean(arrayXYZ, axis=2)
    cdef float xyThreshold = threshold / 100
    
    # Basis matrix:  rows (u,v,w), columns (x,y,z)
    # It's stored as single floats to maximize the chance they'll end up in CPU registers
    # I'm avoiding Numpy since vectorization won't help much with tiny matrices and vectors
    cdef float bux = basis[0,0]
    cdef float buy = basis[0,1]
    cdef float buz = basis[0,2]
    cdef float bvx = basis[1,0]
    cdef float bvy = basis[1,1]
    cdef float bvz = basis[1,2]
    cdef float bwx = basis[2,0]
    cdef float bwy = basis[2,1]
    cdef float bwz = basis[2,2]

    # Voxel coordinates
    cdef int x,y,z

    # Reflectivity at voxel
    cdef float val    

    # Cylindrical coordinates
    cdef float cylU, cylV, cylTheta, cylR
    cdef int cylW, cylThetaIdx, cylRIdx

    # Offset of voxel from the waffle stack's starting point, in xyz basis
    cdef float ox, oy, oz

    # Offset of voxel from waffle center, in uv basis
    cdef float ou, ov

    cdef float radiusRescale = (nRadius-1) / (radiusRange[1]-radiusRange[0])
    cdef float norm
    
    cdef int xStart = min(rangeXYZ[0])
    cdef int xEnd = max(rangeXYZ[0])
    cdef int yStart = min(rangeXYZ[1])
    cdef int yEnd = max(rangeXYZ[1])
    cdef int zStart = min(rangeXYZ[2])
    cdef int zEnd = max(rangeXYZ[2])
    
    # Loop over voxels, accumulate 2D embedding
    for x in range(xStart, xEnd):
        for y in range(yStart, yEnd):

            # Skip empty z-slices
            if xyMask[x,y] < xyThreshold: continue

            for z in range(zStart, zEnd):

                # Skip empty voxels
                val = arrayXYZ[x,y,z]
                if val < threshold: continue

                # Get voxel's offset from starting point, in xyz basis
                ox, oy, oz = (x-startXYZ[0], y-startXYZ[1], z-startXYZ[2])

                # Get axial cylindrical coordinate
                cylW = floor(bwx*ox + bwy*oy + bwz*oz)

                # Ensure that voxel is within the output image's w-range
                if cylW >= 0 and cylW < wLen:

                    # Transform (ox,oy,oz) to uv basis
                    ou, ov = (
                        bux*ox + buy*oy + buz*oz, 
                        bvx*ox + bvy*oy + bvz*oz)

                    # Get voxel's offset from waffle center, in uv basis
                    # An axis-aligned scaling transform is also applied
                    cylU = (ou - uOffsets[cylW]) / (uWidths[cylW])
                    cylV = (ov - vOffsets[cylW]) / (vWidths[cylW])

                    # Get radial cylindrical coordinate
                    cylR = sqrt(cylU**2 + cylV**2)                    

                    # Ensure that voxel is not far from waffle center
                    if cylR >= radiusRange[0] and cylR <= radiusRange[1]:

                        # Get angular cylindrical coordinate
                        cylTheta = atan2(cylU, -cylV)

                        # Get array index from theta
                        cylThetaIdx = floor(nTheta/2 + cylTheta * (nTheta-1) / 2 / 3.14159)
                        cylThetaIdx = 0 if cylThetaIdx < 0 else cylThetaIdx
                        cylThetaIdx = nTheta-1 if cylThetaIdx >= nTheta else cylThetaIdx

                        # Get array index from radius
                        cylRIdx = floor((cylR - radiusRange[0])*radiusRescale)

                        # Accumulate in bin
                        out[cylRIdx, cylThetaIdx, cylW, 0] += 1
                        out[cylRIdx, cylThetaIdx, cylW, 1] += val

    # Normalize by number of voxels
    for cylRIdx in range(nRadius):
        for cylThetaIdx in range(nTheta):
            for cylW in range(wLen):
                norm = out[cylRIdx, cylThetaIdx, cylW, 0]
                if norm > .00000001:
                    out[cylRIdx, cylThetaIdx, cylW, 1] /= norm

    return np.array(out[:,:,:,1], dtype=np.float32)

In [ ]:
trunk['tLen'] = 250
trunk['rLen'] = 120
trunk['highpassRadius'] = 5
trunk['rescaleRadius'] = 0.1
trunk['rescaleReflec'] = 500
trunk['threshold'] = .00015

rightLeg['tLen'] = 80
rightLeg['rLen'] = 40
rightLeg['highpassRadius'] = 5
rightLeg['rescaleRadius'] = 0.1
rightLeg['rescaleReflec'] = 500
rightLeg['threshold'] = .0001

leftLeg['tLen'] = 80
leftLeg['rLen'] = 40
leftLeg['highpassRadius'] = 5
leftLeg['rescaleRadius'] = 0.1
leftLeg['rescaleReflec'] = 500
leftLeg['threshold'] = .0001

rightBicep['tLen'] = 60
rightBicep['rLen'] = 40
rightBicep['highpassRadius'] = (2,5)
rightBicep['rescaleRadius'] = 0.1
rightBicep['rescaleReflec'] = 500
rightBicep['threshold'] = .00001

leftBicep['tLen'] = 60
leftBicep['rLen'] = 40
leftBicep['highpassRadius'] = (2,5)
leftBicep['rescaleRadius'] = 0.1
leftBicep['rescaleReflec'] = 500
leftBicep['threshold'] = .00001

rightForearm['tLen'] = 30
rightForearm['rLen'] = 35
rightForearm['highpassRadius'] = (2,5)
rightForearm['rescaleRadius'] = 0.1
rightForearm['rescaleReflec'] = 500
rightForearm['threshold'] = .00001

leftForearm['tLen'] = 30
leftForearm['rLen'] = 35
leftForearm['highpassRadius'] = (2,5)
leftForearm['rescaleRadius'] = 0.1
leftForearm['rescaleReflec'] = 500
leftForearm['threshold'] = .00001

In [ ]:
# Simulated arguments to getSegmentImage()

segment = rightForearm
dat = data
rRange = np.array([0.2,2], dtype=np.float32)
highPassRadius = segment['highpassRadius']
threshold = segment['threshold']
rescaleRadius = segment['rescaleRadius']
rescaleReflec = segment['rescaleReflec']
tLen = segment['tLen']
rLen = segment['rLen']

In [ ]:
####################################################
# Embed segment in cylindrical coords.  (Place in getSegmentImage)
####################################################

#%time \
cylindrical = toCylindricalCoordinates( \
    threshold, dat, \
    segment['rangeXYZ'], segment['basis'], segment['start'], \
    segment['uOffsets'], segment['uWidths'], \
    segment['vOffsets'], segment['vWidths'], \
    tLen, rLen, \
    np.array(rRange, dtype=np.float32) \
)

In [ ]:
fig, ax = plt.subplots(3, figsize=(30,30))
for i in range(3):
    ax[i].imshow(np.take(cylindrical, math.floor(cylindrical.shape[i]/2), axis=i), cmap = 'viridis', interpolation = 'nearest')

# Pad image on theta axis

In [ ]:
def pad(arr, portionOfCircle):
    
    nTheta = arr.shape[0]
    nPad = math.floor(nTheta * portionOfCircle)
    
    out = np.array(arr, dtype=np.float32)
    out = np.concatenate([out[nTheta-nPad:nTheta], out, out[0:nPad]], axis=0)
    
    return out

In [ ]:
####################################################
# Pad image on theta axis.  (Place in getSegmentImage)
####################################################
cylindrical = np.transpose(cylindrical, axes=(1,0,2))
cylindrical = pad(cylindrical, .25)
cylindrical = np.transpose(cylindrical, axes=(1,0,2))

In [ ]:
fig, ax = plt.subplots(3, figsize=(30,30))
for i in range(3):
    ax[i].imshow(np.take(cylindrical, math.floor(cylindrical.shape[i]/2), axis=i), cmap = 'viridis', interpolation = 'nearest')

# Smooth and inpaint to remove Moire gaps

In [ ]:
%%cython 
#--annotate
cimport cython
import math
import numpy as np
from cython.view cimport array as cvarray
    
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)

def boxFilterNonzero(float [:,:,:] array):

    """
    Convolutional filter that gets the average of non-zero neighbors
    """
    
    cdef int nr = array.shape[0]
    cdef int nt = array.shape[1]
    cdef int nw = array.shape[2]

    cdef float [:,:,:] out = array.copy()

    cdef int r, t, w, dr, dt, dw, n
    cdef float dest, source, total
    
    for r in range(1, nr-1):
        for t in range(1, nt-1):
            for w in range(1, nw-1):

                dest = array[r,t,w]

                if dest < 0.00000001:

                    n = 0
                    total = 0

                    for dr in range(-1,1):
                        for dt in range(-1,1):
                            for dw in range(-1,1):
                                source = array[r+dr,t+dt,w+dw]
                                if source > 0.00000001:
                                    n += 1
                                    total += source

                    if n > 0:
                        total /= n
                        out[r,t,w] = total
                        
    return np.array(out, dtype=np.float32)            

In [ ]:
%%cython 
#--annotate
cimport cython
import math
import numpy as np
from cython.view cimport array as cvarray
    
@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def inpaint(float [:,:,:] unsmoothed, float [:,:,:] smoothed, float [:,:,:] smoothedPresence):
    """
    Inpaints an image by replacing zeros in the image with values taken from a smoothed image.
    It's much faster than scipy's inpainting function.
    """
    
    cdef int nr = unsmoothed.shape[0]
    cdef int nt = unsmoothed.shape[1]
    cdef int nw = unsmoothed.shape[2]
    
    cdef float [:,:,:] out = unsmoothed.copy()
    cdef int r, t, w
    
    for r in range(nr):
        for t in range(nt):
            for w in range(nw):
                
                if unsmoothed[r,t,w] < .00000001 and smoothedPresence[r,t,w] > 0.01:
                    out[r,t,w] = smoothed[r,t,w]/smoothedPresence[r,t,w]

    return out

In [ ]:
####################################################
# Filter and inpaint to remove Moire gaps.  (Place in getSegmentImage)
####################################################
boxFiltered = boxFilterNonzero(cylindrical)
smoothed = sp.ndimage.filters.gaussian_filter(boxFiltered, (1,2,2))
presence = np.array(boxFiltered > 0.00000001, dtype=np.float32)
smoothedPresence = sp.ndimage.filters.gaussian_filter(presence, (1,2,2))
inpainted = inpaint(boxFiltered, smoothed, smoothedPresence)

In [ ]:
fig, ax = plt.subplots(3, figsize=(30,30))
for i in range(3):
    ax[i].imshow(np.take(inpainted, math.floor(inpainted.shape[i]/2), axis=i), cmap = 'viridis', interpolation = 'nearest')

smoothed = sp.ndimage.filters.median_filter(cylindrical, (2,4,4))
smoothed = sp.ndimage.filters.gaussian_filter(smoothed, (1,2,2))
inpainted = inpaint(cylindrical, smoothed)

# Get surface moments

In [ ]:
%%cython 
#--annotate
cimport cython
import math
import numpy as np
from cython.view cimport array as cvarray

cdef extern from "math.h":
    int floor(float x)

@cython.boundscheck(False) # turn off bounds-checking for entire function
@cython.wraparound(False)  # turn off negative index wrapping for entire function
@cython.cdivision(True)
def surfaceMoments(float [:,:,:] array, float thresholdPortion):
    """
    Given an array with dimensions (r, theta, w), this function
    calculates the peak moments in the r lists, and returns
    a 2+1 dimensional array with moments 0, 1, and 2 as image channels.    
    """
    
    cdef int nr = array.shape[0]
    cdef int nt = array.shape[1]
    cdef int nw = array.shape[2]
    
    cdef float [:,:,:] out = np.zeros(shape=(nt,nw,3), dtype=np.float32)
    
    cdef float val, maxVal, maxPos
    cdef int r, windowStart, windowEnd
    cdef float total0, total1, total2
    cdef float threshold, subtract

    for w in range(nw):
        for t in range(nt):

            # Get maximum
            maxVal = -1
            maxPos = 0
            for r in range(nr):
                val = array[r,t,w]
                if val > maxVal:
                    maxVal = val
                    maxPos = r

            # Improve peak position
            windowStart = floor(maxPos) - 2
            windowStart = 0 if windowStart<0 else windowStart

            windowEnd = windowStart + 4
            windowEnd = nr if windowEnd>nr else windowEnd

            total0 = 0
            total1 = 0
            for r in range(windowStart, windowEnd):
                total0 += array[r,t,w]
                total1 += array[r,t,w] * r
            if total0 > .00000001:
                maxPos = total1 / total0

            # Get a threshold for the next part
            threshold = maxVal * thresholdPortion
            subtract = threshold # set this to zero for flatter mass distribution

            # Get spatial moment 0, using thresholded masses
            total0 = 0
            total1 = 0
            for r in range(nr):
                val = array[r,t,w]
                if val >= threshold:
                    val -= subtract
                    total0 += val

            if total0 > 0.00000001:

                # Get spatial moment 2, using thresholded masses
                total2 = 0
                for r in range(nr):
                    val = array[r,t,w]
                    if val >= threshold:
                        val -= subtract
                        total2 += val * (r - maxPos) ** 2
                total2 = (total2 / total0) ** 0.5

            out[t,w,0] = maxVal
            out[t,w,1] = maxPos
            out[t,w,2] = total2

    return np.array(out, dtype=np.float32)

In [ ]:
####################################################
# Get surface moments.  (Place in getSegmentImage)
####################################################
moments = surfaceMoments(inpainted, 0.5)

In [ ]:
fig, ax = plt.subplots(3, figsize=(30,30))
for i in range(3):
    ax[i].imshow(moments[:,:,i])

# Rescale image channels

In [ ]:
def imageAdjust(img, scale):
    out = img.copy()
    for i in range(img.shape[2]):
        median = np.median(img[:,:,i])
        out[:,:,i] = out[:,:,i] - median
        out[:,:,i] *= scale[i]
        out[:,:,i] += 0.5
        out[:,:,i] = np.clip(out[:,:,i], 0, 1)
    return out

In [ ]:
####################################################
# Rescale moments to (0,1).  (Place in getSegmentImage)
####################################################
img = imageAdjust(moments, (1000, 0.01, 0.05))

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(img)

# Depad/repad image

In [ ]:
def depad(arr, portion):
    nTheta = len(arr)
    nPad = math.floor(nTheta * portion)
    out = arr[nPad:-nPad]
    return out

In [ ]:
####################################################
# Depad/repad image.  (Place in getSegmentImage)
####################################################
img = depad(img, .1666)
img = pad(img, .25)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(img)

# Get image for a segment

In [ ]:
def getSegmentImage(
    threshold,
    dat,
    segment,
    tLen,
    rLen,
    rRange,
    highpassRadius,
    rescaleRadius,
    rescaleReflec,
    rescaleThickness):
    
    global threshold3D

    ####################################################
    # Embed segment in cylindrical coords.  (Place in getSegmentImage)
    ####################################################
    cylindrical = toCylindricalCoordinates( \
        threshold, dat, \
        segment['rangeXYZ'], segment['basis'], segment['start'], \
        segment['uOffsets'], segment['uWidths'], \
        segment['vOffsets'], segment['vWidths'], \
        tLen, rLen, \
        np.array(rRange, dtype=np.float32) \
    )

    ####################################################
    # Pad image on theta axis.  (Place in getSegmentImage)
    ####################################################
    cylindrical = np.transpose(cylindrical, axes=(1,0,2))
    cylindrical = pad(cylindrical, .25)
    cylindrical = np.transpose(cylindrical, axes=(1,0,2))

    ####################################################
    # Filter and inpaint to remove Moire gaps.  (Place in getSegmentImage)
    ####################################################
    boxFiltered = boxFilterNonzero(cylindrical)
    smoothed = sp.ndimage.filters.gaussian_filter(boxFiltered, (1,2,2))
    presence = np.array(boxFiltered > 0.00000001, dtype=np.float32)
    smoothedPresence = sp.ndimage.filters.gaussian_filter(presence, (1,2,2))
    inpainted = inpaint(boxFiltered, smoothed, smoothedPresence)

    ####################################################
    # Get surface moments.  (Place in getSegmentImage)
    ####################################################
    moments = surfaceMoments(inpainted, 0.5)
    
    ####################################################
    # Rescale moments to (0,1).  (Place in getSegmentImage)
    ####################################################
    img = imageAdjust(moments, (rescaleReflec, rescaleRadius, rescaleThickness))

    ####################################################
    # Depad/repad image.  (Place in getSegmentImage)
    ####################################################
    img = depad(img, .1666)
    img = pad(img, .25)

    return img

# Get trunk image

The remaining sections are to be placed in the main loop, not in getSegmentImage() like the last few sections.

In [ ]:
####################################################
log('Get trunk image')
####################################################
trunkImg = getSegmentImage(
    threshold = .00015,
    dat = data,
    segment = trunk,
    tLen = 250,
    rLen = 120,
    rRange = (0.2,2),
    highpassRadius = 5,
    rescaleReflec = 1500,
    rescaleRadius = .03,
    rescaleThickness = .13
)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(trunkImg)

# Right leg

In [ ]:
####################################################
log('Get right leg image')
####################################################
rightLegImg = getSegmentImage(
    threshold = .0001,
    dat = data,
    segment = rightLeg,
    tLen = 100,
    rLen = 80,
    rRange = (0.2,2),
    highpassRadius = 5,
    rescaleReflec = 1500,
    rescaleRadius = .03,
    rescaleThickness = .133
)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(rightLegImg)

# Left leg

In [ ]:
####################################################
log('Get left leg image')
####################################################
leftLegImg = getSegmentImage(
    threshold = .0001,
    dat = data,
    segment = leftLeg,
    tLen = 100,
    rLen = 80,
    rRange = (0.2,2),
    highpassRadius = 5,
    rescaleReflec = 1500,
    rescaleRadius = .03,
    rescaleThickness = .133
)
leftLegImg = np.flipud(leftLegImg)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(leftLegImg)

# Right bicep

In [ ]:
####################################################
log('Get right bicep image')
####################################################
rightBicepImg = getSegmentImage(
    threshold = .00001,
    dat = data,
    segment = rightBicep,
    tLen = 60,
    rLen = 40,
    rRange = (0.2,3),
    highpassRadius = (2,5),
    rescaleReflec = 1500,
    rescaleRadius = .03,
    rescaleThickness = .133
)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(rightBicepImg)

# Left bicep

In [ ]:
####################################################
log('Get left bicep image')
####################################################
leftBicepImg = getSegmentImage(
    threshold = .00001,
    dat = data,
    segment = leftBicep,
    tLen = 60,
    rLen = 40,
    rRange = (0.2,3),
    highpassRadius = (2,5),
    rescaleReflec = 1500,
    rescaleRadius = .03,
    rescaleThickness = .133
)
leftBicepImg = np.flipud(leftBicepImg)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(leftBicepImg)

# Right forearm

In [ ]:
####################################################
log('Get right forearm image')
####################################################
rightForearmImg = getSegmentImage(
    threshold = .00001,
    dat = data,
    segment = rightForearm,
    tLen = 50,
    rLen = 35,
    rRange = (0.2,3),
    highpassRadius = (2,5),
    rescaleReflec = 1500,
    rescaleRadius = .02,
    rescaleThickness = .133
)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(rightForearmImg)

# Left forearm

In [ ]:
####################################################
log('Get left forearm image')
####################################################
leftForearmImg = getSegmentImage(
    threshold = .00001,
    dat = data,
    segment = leftForearm,
    tLen = 50,
    rLen = 35,
    rRange = (0.2,3),
    highpassRadius = (2,5),
    rescaleReflec = 1500,
    rescaleRadius = .02,
    rescaleThickness = .133
)
leftForearmImg = np.flipud(leftForearmImg)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(leftForearmImg)

# Combine images

In [ ]:
####################################################
log('Combine images')
####################################################
trunkResized = resize(trunkImg, (360,256), mode='edge')
rightLegResized = resize(rightLegImg, (180,256), mode='edge')
leftLegResized = resize(leftLegImg, (180,256), mode='edge')
rightBicepResized = resize(rightBicepImg, (90,128), mode='edge')
leftBicepResized = resize(leftBicepImg, (90,128), mode='edge')
rightForearmResized = resize(rightForearmImg, (70,128), mode='edge')
leftForearmResized = resize(leftForearmImg, (70,128), mode='edge')

forearmEmpty = np.full(shape=(20,128,3), fill_value=.5)
rightForearmResized = np.concatenate([rightForearmResized, forearmEmpty], axis=0)
leftForearmResized = np.concatenate([leftForearmResized, forearmEmpty], axis=0)

rightArm = np.concatenate([rightBicepResized, rightForearmResized], axis=1)
leftArm = np.concatenate([leftBicepResized, leftForearmResized], axis=1)

bodyImg = np.concatenate([
    trunkResized,
    rightLegResized,
    leftLegResized,
    rightArm,
    leftArm
    ], axis=0)

In [ ]:
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(bodyImg)

# Save image

In [ ]:
outputDir = embeddedDir1

In [ ]:
####################################################
log('Save image')
####################################################
base = inputFile.split('/')[-1]
base = base.split('.')[-2]
outputFile = outputDir[6:] + base + '.png'
imsave(outputFile, bodyImg)

# Function to process a scan (all of the above copy/pasted here)

In [ ]:
def processScan(inputFile, outputDir):

  ####################################################
  log('+++++ Read file ' + inputFile + ' +++++')
  ####################################################
  # Note: title comments like the one above indicate 
  # notebook cells that are copy/pasted to the main 
  # processing loop at the bottom of the notebook.
  data = loadFile(inputFile)

  ####################################################
  log('Threshold')
  ####################################################
  thresholded = np.clip(data, threshold3D, 100000) - threshold3D

  ####################################################
  log('Average along axii to get 2D mean images')
  ####################################################
  # Summed images will have yz, xz, xy axes, respectively
  means2D = list([thresholded.mean(axis=i) for i in range(3)])
  mean2DAxii = [('y', 'z'), ('x','z'), ('x','y')]

  ####################################################
  log('Get center of mass')
  ####################################################
  # Could do this on the 3D image, but it's much faster on 2D images 
  xCenter, zCenter = np.round(ndimage.measurements.center_of_mass(means2D[1])).astype(int)
  yCenter, dmy = np.round(ndimage.measurements.center_of_mass(means2D[0])).astype(int)

  ####################################################
  log('Get end of trunk z-range (top of shoulders)')
  ####################################################
  zStart, zEnd = (int(zCenter * 1.2), int(zCenter * 1.7))
  if zEnd > data.shape[2]: zEnd = data.shape[2]

  xStart = int(xCenter - .25 * zCenter)
  xEnd = int(xCenter + .25 * zCenter)

  midStrip = means2D[1][xStart:xEnd, zStart:zEnd]

  off = midStrip < 0.00000001
  numOff = np.sum(off, axis=0)

  trunkEnd = firstOn(numOff, .15)
  trunkEnd += zStart

  ####################################################
  log('Get start of trunk z-range (bottom of groin)')
  ####################################################
  zStart = int(0.5 * zCenter)
  zStrip = np.mean(thresholded[:, yCenter:, zStart:zCenter], axis=0)
  zStrip = gaussian_filter(zStrip, (5,5))
  buttProfile = np.array([firstOff(x, .05) for x in np.transpose(zStrip)])
  buttProfile = median_filter(buttProfile, 5)
  buttProfile = savgol_filter(buttProfile, 11, 2, deriv=1, mode='nearest')
  trunkStart = zStart + np.argmax(buttProfile)

  ####################################################
  log('Erase head')
  ####################################################
  # Head position, size
  headCenterZ = trunkEnd * 1.12
  headHeight = trunkEnd * .15
  headWidth = trunkEnd * .13

  # Get center X for head, which may be shifted relative to body
  headStripStartZ = math.floor(trunkEnd * 1.05)
  headStripEndZ = math.floor(trunkEnd * 1.1)
  headStripStartX = math.floor(xCenter - trunkEnd*.18)
  headStripEndX = math.floor(xCenter + trunkEnd*.18)
  headStripMean = np.mean(thresholded[headStripStartX:headStripEndX, :, headStripStartZ:headStripEndZ], axis=(1,2))
  middle = math.floor(len(headStripMean)/2)
  rightMost = firstOff(headStripMean[middle:0:-1], .05)
  leftMost = firstOff(headStripMean[middle:-1], .05)
  headOffsetX = math.floor((leftMost-rightMost)/2)

  # Erase head by deleting cylindrical region
  thresholded = eraseHead(thresholded, xCenter + headOffsetX, headCenterZ, headWidth, headHeight)
  data = eraseHead(data, xCenter + headOffsetX, headCenterZ, headWidth, headHeight)

  # Recalculate averages along axii
  means2D = list([thresholded.mean(axis=i) for i in range(3)])
  mean2DAxii = [('y', 'z'), ('x','z'), ('x','y')]

  ####################################################
  log('Get leg and trunk waffle stacks')
  ####################################################

  # Right leg
  end = int(1.27 * trunkStart)
  rightLeg = makeWaffles(
      thresholded, 
      (0, xCenter), 
      (0, thresholded.shape[1]), 
      (0, end), 
      0, end, np.eye(3), 200, 0.00001, .16
  )

  # Left leg
  end = int(1.27 * trunkStart)
  leftLeg = makeWaffles(
      thresholded, 
      (xCenter, thresholded.shape[0]), 
      (0, thresholded.shape[1]), 
      (0, end), 
      0, end, np.eye(3), 200, 0.00001, .16
  )

  # Trunk
  start = int(.95 * trunkStart)
  end = int(trunkEnd)
  trunk = makeWaffles(
      thresholded, 
      (0, thresholded.shape[0]), 
      (0, thresholded.shape[1]), 
      (start, end), 
      start, end, np.eye(3), 250, 0.00001, .16
  )

  ####################################################
  log('Get arm waffle stacks')
  ####################################################

  # Guess armpit height
  armpitZ = int(trunkStart + 0.9 * (trunkEnd - trunkStart))

  # Right arm
  shoulderPos, elbowPos, wristPos, fingertipZ = getArmJointCoordinates(thresholded, means2D, xCenter, armpitZ, False)

  # Right bicep
  rightBicep = makeWaffles(
      thresholded,
      (0, xCenter), (0, thresholded.shape[1]), (math.floor(shoulderPos[2]*.9), elbowPos[2]+20),
      shoulderPos, elbowPos, getArmSegmentBasis(shoulderPos, elbowPos), 80, 0.00001, .7
  )

  # Right forearm
  rightForearm = makeWaffles(
      thresholded,
      (0, xCenter), (0, thresholded.shape[1]), (elbowPos[2]-20, wristPos[2]+20),
      elbowPos, wristPos, getArmSegmentBasis(elbowPos, wristPos), 60, 0.00001, .9
  )

  # Left arm
  shoulderPos, elbowPos, wristPos, fingertipZ = getArmJointCoordinates(thresholded, means2D, xCenter, armpitZ, True)

  # Left bicep
  leftBicep = makeWaffles(
      thresholded,
      (xCenter, thresholded.shape[0]), (0, thresholded.shape[1]), (math.floor(shoulderPos[2]*.9), elbowPos[2]+20),
      shoulderPos, elbowPos, getArmSegmentBasis(shoulderPos, elbowPos), 80, 0.00001, .7
  )

  # Left forearm
  leftForearm = makeWaffles(
      thresholded,
      (xCenter, thresholded.shape[0]), (0, thresholded.shape[1]), (elbowPos[2]-20, wristPos[2]+20),
      elbowPos, wristPos, getArmSegmentBasis(elbowPos, wristPos), 60, 0.00001, .9
  )

  ####################################################
  log('Get trunk image')
  ####################################################
  trunkImg = getSegmentImage(
      threshold = .00015,
      dat = data,
      segment = trunk,
      tLen = 250,
      rLen = 120,
      rRange = (0.2,2),
      highpassRadius = 5,
      rescaleReflec = 1500,
      rescaleRadius = .03,
      rescaleThickness = .13
  )

  ####################################################
  log('Get right leg image')
  ####################################################
  rightLegImg = getSegmentImage(
      threshold = .0001,
      dat = data,
      segment = rightLeg,
      tLen = 100,
      rLen = 80,
      rRange = (0.2,2),
      highpassRadius = 5,
      rescaleReflec = 1500,
      rescaleRadius = .03,
      rescaleThickness = .133
  )

  ####################################################
  log('Get left leg image')
  ####################################################
  leftLegImg = getSegmentImage(
      threshold = .0001,
      dat = data,
      segment = leftLeg,
      tLen = 100,
      rLen = 80,
      rRange = (0.2,2),
      highpassRadius = 5,
      rescaleReflec = 1500,
      rescaleRadius = .03,
      rescaleThickness = .133
  )
  leftLegImg = np.flipud(leftLegImg)

  ####################################################
  log('Get right bicep image')
  ####################################################
  rightBicepImg = getSegmentImage(
      threshold = .00001,
      dat = data,
      segment = rightBicep,
      tLen = 60,
      rLen = 40,
      rRange = (0.2,3),
      highpassRadius = (2,5),
      rescaleReflec = 1500,
      rescaleRadius = .03,
      rescaleThickness = .133
  )

  ####################################################
  log('Get left bicep image')
  ####################################################
  leftBicepImg = getSegmentImage(
      threshold = .00001,
      dat = data,
      segment = leftBicep,
      tLen = 60,
      rLen = 40,
      rRange = (0.2,3),
      highpassRadius = (2,5),
      rescaleReflec = 1500,
      rescaleRadius = .03,
      rescaleThickness = .133
  )
  leftBicepImg = np.flipud(leftBicepImg)

  ####################################################
  log('Get right forearm image')
  ####################################################
  rightForearmImg = getSegmentImage(
      threshold = .00001,
      dat = data,
      segment = rightForearm,
      tLen = 50,
      rLen = 35,
      rRange = (0.2,3),
      highpassRadius = (2,5),
      rescaleReflec = 1500,
      rescaleRadius = .02,
      rescaleThickness = .133
  )

  ####################################################
  log('Get left forearm image')
  ####################################################
  leftForearmImg = getSegmentImage(
      threshold = .00001,
      dat = data,
      segment = leftForearm,
      tLen = 50,
      rLen = 35,
      rRange = (0.2,3),
      highpassRadius = (2,5),
      rescaleReflec = 1500,
      rescaleRadius = .02,
      rescaleThickness = .133
  )
  leftForearmImg = np.flipud(leftForearmImg)

  ####################################################
  log('Combine images')
  ####################################################
  trunkResized = resize(trunkImg, (360,256), mode='edge')
  rightLegResized = resize(rightLegImg, (180,256), mode='edge')
  leftLegResized = resize(leftLegImg, (180,256), mode='edge')
  rightBicepResized = resize(rightBicepImg, (90,128), mode='edge')
  leftBicepResized = resize(leftBicepImg, (90,128), mode='edge')
  rightForearmResized = resize(rightForearmImg, (70,128), mode='edge')
  leftForearmResized = resize(leftForearmImg, (70,128), mode='edge')

  forearmEmpty = np.full(shape=(20,128,3), fill_value=.5)
  rightForearmResized = np.concatenate([rightForearmResized, forearmEmpty], axis=0)
  leftForearmResized = np.concatenate([leftForearmResized, forearmEmpty], axis=0)

  rightArm = np.concatenate([rightBicepResized, rightForearmResized], axis=1)
  leftArm = np.concatenate([leftBicepResized, leftForearmResized], axis=1)

  bodyImg = np.concatenate([
      trunkResized,
      rightLegResized,
      leftLegResized,
      rightArm,
      leftArm
      ], axis=0)

  ####################################################
  log('Save image')
  ####################################################
  base = inputFile.split('/')[-1]
  base = base.split('.')[-2]
  outputFile = outputDir[6:] + base + '.png'
  imsave(outputFile, bodyImg)

# Function to process a folder of scans

In [ ]:
def processFolder(inputFolder, outputFolder, startFileNum=0):

  inputFiles = filenames(inputFolder)

  for fileNum in range(startFileNum, len(inputFiles)):
      
      # Print file number (don't just freeze for hours)
      print(str(datetime.datetime.now()) + "    Processing file " + str(fileNum) + " of " + str(len(inputFiles)))

      try:
        processScan(inputFiles[fileNum], outputFolder)

      except Exception as exc:
          errorStr = 'Error: ' + str(exc)
          log(errorStr)
          print(errorStr)
          break

  log("Finished")
  print(str(datetime.datetime.now()) + "    Finished")

# Process scans

Manually clear all output now so that autosave doesn't waste CPU cycles.  I didn't find a way to do this in code.

In [ ]:
# Process stage 1 scans
# Takes about 15 hours
processFolder(scanDir1, embeddedDir1, 0)

In [ ]:
# Process stage 2 scans
# Takes about 15 hours
processFolder(scanDir2, embeddedDir2, 0)

# Do we have them all?

In [ ]:
inputFiles = filenames(scanDir1)

In [ ]:
inputFiles = set([x[17:-4] for x in inputFiles])

In [ ]:
outputFiles = filenames(embeddedDir1)

In [ ]:
outputFiles = set([x[24:-4] for x in outputFiles if x[-3:] == 'png'])

In [ ]:
print("length: ", len(outputFiles), len(inputFiles))
print("same length?  ", len(outputFiles)==len(inputFiles))
print("difference:  ", outputFiles.symmetric_difference(inputFiles))

# Compare scans and embeddings

In [ ]:
inputFile = filenames(scanDir1)[100]

In [ ]:
# Load scan
data = loadFile(inputFile)
thresholded = np.clip(data, threshold3D, 100000) - threshold3D

# Plot scan
fig, ax = plt.subplots(3, figsize=(30,30))
for i in range(3):
    ax[i].imshow(np.mean(thresholded, axis=i), cmap = 'viridis', interpolation = 'nearest')

In [ ]:
# Load embedding
png = loadFile(embeddedDir1 + inputFile[17:-4] + '.png')

# Plot embedding
fig, ax = plt.subplots(1, figsize=(30,30))
ax.imshow(png)